In [9]:

import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split 
import pandas as pd
import os
import glob

In [10]:
dat= r"C:\Users\pawel\Desktop\data"

exeel_fille= glob.glob(os.path.join(dat,"**","*.xlsm"), recursive=True)
fille=[]

In [11]:

for exel in exeel_fille:
    ex=pd.read_excel(exel, sheet_names="read")
    if not ex.empty:
        fille.append(ex)

In [12]:
data=pd.concat(fille, ignore_index=True)

data=data.dropna()
data=data[["accelerometer_X","accelerometer_Y","accelerometer_Z"]].values

ValueError: No objects to concatenate

In [13]:
scalar=StandarScalar()
x=scalar.fit_transform(X)

NameError: name 'StandarScalar' is not defined

In [14]:
X_train, X_test, Y_train, Y_test, Z_train, Z_test= train_test_split(accelerometer_X,accelerometer_Y,accelerometer_Z, test_size=0.4,random_state=40)

NameError: name 'accelerometer_X' is not defined

In [15]:
model_Y=SVC(kernel="rbf", C=1,gamma="auto")
model_Y.fit(X_train,Y_train)
model_Z=SVC(kernel="rbf",C=1,gamma="auto")
meodel_Z.fit(X_train, Z_train)
print(model_Y)
print(model_Z)

NameError: name 'X_train' is not defined